This module is dedicated for traning the computer to play Tic Tac Toe game. Computer take turns first identified as 'max'. After that other agent will take turn as 'min'.

First of all we need to make the game tree. After that we need to keep track on each terminal state, it will be used for training by minimax.
We have these 4 methods for identifying the terminal state.
1) isVerticalMatch(state) : Identifies whether this state has vertical match or not
2) isHorizontalMatch(state) : Tests if this state has horizontal match or not
3) isDiagonalMatch(state) : Shows whether this state has diagonal match or not
4) idGameTreeEnd(state) : Returns True if state is the leaf node in Game Tree



In [1]:
import random
import json

In [2]:
terminalStates = []     # keeps track on the terminal state of this game tree

class State:
    parentDict = {}

    def __init__(self, parent, data, level, children):
        self.parent = parent
        self.data = data
        self.level = level
        self.childrens = children
        self.score = 0

    def addChildren(self, player):
        if not self.isTerminalState():
            entry = 'X' if player == 'max' else 'O'
            for i in range(9):
                if self.data[i] == 'E':
                    # make a list with a new entry at ith position
                    copy = self.data[:]
                    copy[i] = entry
                    # append this to childrens List
                    childLevel = self.level + 1
                    currentChild = State(self, copy, childLevel, [])
                    self.childrens.append(currentChild)
        else:
            parent = self.parent                # to keep track on the terminal state
            if parent not in State.parentDict:  # for inserting the one node only one times
                terminalStates.append(self)     # we need append those elements only whose parents are not same
            State.parentDict[parent] = 1        # for storing unique parent, we are using parent dict

    def makeGameTree(self):                     # recursive method to make the whole game tree
        depth = self.level
        turn = 'max' if depth % 2 == 0 else 'min'
        if depth <= 9:
            self.addChildren(turn)
            for child in self.childrens:
                child.makeGameTree()

    # methods for adding score and checking if its the terminal state or not
    def isVerticalMatch(self):
        '''
        returns True if there is vertical matching in data else False
        '''
        lst = self.data
        for i in range(3):
            lst1 = []
            for j in range(3):
                value = lst[i+3*j]
                lst1.append(value)
            s = list(set(lst1))
            if len(s) == 1:
                if s[0] == 'O':
                    self.score = -1
                    return 'min'
                elif s[0] == 'X':
                    self.score = 1
                    return 'max'
        # self.score = 0
        return False

    def isHorizontalMatch(self):
        lst = self.data
        for i in range(3):
            lst1 = []
            for j in range(3):
                value = lst[3*i+j]
                lst1.append(value)
            s = list(set(lst1))
            if len(s) == 1:
                if s[0] == 'O':
                    self.score = -1
                    return 'min'
                elif s[0] == 'X':
                    self.score = 1
                    return 'max'

        # self.score = 0
        return False


    def isDiagonalMatch(self):
        lst= self.data
        t1 = list(set([lst[0], lst[4], lst[8]]))
        t2 = list(set([lst[2], lst[4], lst[6]]))
        if len(t1) == 1:
            if t1[0] == 'O':
                self.score = -1
                return 'min'
            elif t1[0] == 'X':
                self.score = 1
                return 'max'

        if len(t2) == 1:
            if t2[0] == 'O':
                self.score = -1
                return 'min'
            elif t2[0] == 'X':
                self.score = 1
                return 'max'

        # self.score = 0
        return False


    def isLevelEnd(self):
        depth = self.level
        if depth == 9:
            self.score = 0
            return "Tie"
        return False


    def isTerminalState(self):
        '''
        returns winning player or "Tie" if state is a terminal state else False

        state is a state type object representing one node in game tree
        '''
        condition = self.isVerticalMatch() or self.isHorizontalMatch() or self.isDiagonalMatch() or self.isLevelEnd()
        return condition


In [3]:
start = ['E' for i in range(9)]
Gametree = [start]
zeroState = State(None, start, 0, [])
zeroState.score = 0
zeroState.makeGameTree()        # this will make the whole game tree


In [4]:
# Now training gametree with minimax
def trainMinimax(gametree):
    # gametree refers to list of all the terminal states
    for state in gametree:      # starting from terminal nodes
        if state.parent:
            parent = state.parent
            scoreList = [child.score for child in parent.childrens]
            stateLevel = state.level
            turn = 'max' if stateLevel % 2 == 0 else 'min'
            if turn == 'max':
                score = min(scoreList)
            else:
                score = max(scoreList)
            parent.score = score
            gametree.append(parent) # dikkat ye hai ki parent multiple time append ho jayega


trainMinimax(terminalStates)

In [5]:
# getting all the states and its score in a json after training
dict = {}
def makeDict(state):                     # recursive method to make the whole game tree
    depth = state.level
    if depth <= 9:
        for child in state.childrens:
            dict[str(child.data)] = child.score
            makeDict(child)

In [6]:
makeDict(zeroState)
with open(r"C:\Users\LENOVO-PC\Downloads\gameTree.json", 'w') as json_file:
    json.dump(dict, json_file, indent = 4)

In [7]:
def compMove(state):
    parentScore = state.score
    for child in state.childrens:
        # print(child.score, parentScore)
        if child.score == parentScore:
            return child

def humanMove(state, index):
    Temp = state.data[:]
    Temp[index] = 'O'
    for child in state.childrens:
        if child.data == Temp:
            return child

def displayState(state):
    print('###################')
    print('| ', state[0], ' | ', state[1], ' | ', state[2], ' |')
    print('-------------------')
    print('| ', state[3], ' | ', state[4], ' | ', state[5], ' |')
    print('-------------------')
    print('| ', state[6], ' | ', state[7], ' | ', state[8], ' |')
    print('###################')

In [8]:
def play():
    nextState = compMove(zeroState)

    for i in range(4):
        displayState(nextState.data)
        index = int(input('Enter your move from 0 to 8 '))
        nextState = humanMove(nextState, index)
        if nextState is None:
          return 'Enter valid index'
        value = nextState.isTerminalState()
        # print(nextState.data, value)
        if  value:
          displayState(nextState.data)
          return value
        # displayState(nextState.data)
        nextState = compMove(nextState)
        val = nextState.isTerminalState()
        if val:
          displayState(nextState.data)
          return val

    return "Tie"



In [9]:
play()

###################
|  X  |  E  |  E  |
-------------------
|  E  |  E  |  E  |
-------------------
|  E  |  E  |  E  |
###################
###################
|  X  |  E  |  X  |
-------------------
|  E  |  E  |  O  |
-------------------
|  E  |  E  |  E  |
###################
###################
|  X  |  O  |  X  |
-------------------
|  E  |  X  |  O  |
-------------------
|  E  |  E  |  E  |
###################
###################
|  X  |  O  |  X  |
-------------------
|  E  |  X  |  O  |
-------------------
|  X  |  E  |  O  |
###################


'max'

Training by reinforcement learning

In [10]:
start = ['E' for i in range(9)]
Gametree = [start]
zeroState1 = State(None, start, 0, [])
zeroState1.makeGameTree()
alpha = 0.3     # learning rate

In [11]:
def treeTraining(state):
    if state.isTerminalState():
        return state.score
    childrenCount = len(state.childrens)
    a = random.randint(0, childrenCount-1)
    # print(a)
    child = state.childrens[a]
    state.score = state.score + alpha*(child.score - state.score)
    # print(state.score)
    return treeTraining(child)

def reinforcementLearning():
    for i in range(100000):
        (treeTraining(zeroState1))

In [12]:
reinforcementLearning()

In [13]:
def compMove(state):
    scoreList = [child.score for child in state.childrens]
    maxScore = max(scoreList)
    for child in state.childrens:
        # print(child.score, parentScore)
        if child.score == maxScore:
            return child

def humanMove(state, index):
    Temp = state.data[:]
    Temp[index] = 'O'
    for child in state.childrens:
        if child.data == Temp:
            return child

def displayState(state):
    print('###################')
    print('| ', state[0], ' | ', state[1], ' | ', state[2], ' |')
    print('-------------------')
    print('| ', state[3], ' | ', state[4], ' | ', state[5], ' |')
    print('-------------------')
    print('| ', state[6], ' | ', state[7], ' | ', state[8], ' |')
    print('###################')

In [14]:
def play():
    nextState = compMove(zeroState1)

    for i in range(4):
        displayState(nextState.data)
        index = int(input('Enter your move from 0 to 8 '))
        nextState = humanMove(nextState, index)
        if nextState is None:
          return 'Enter valid index'
        value = nextState.isTerminalState()
        # print(nextState.data, value)
        if  value:
          displayState(nextState.data)
          return value
        # displayState(nextState.data)
        nextState = compMove(nextState)
        val = nextState.isTerminalState()
        if val:
          displayState(nextState.data)
          return val

    return "Tie"



In [15]:
play()

###################
|  E  |  E  |  E  |
-------------------
|  E  |  X  |  E  |
-------------------
|  E  |  E  |  E  |
###################
###################
|  X  |  O  |  E  |
-------------------
|  E  |  X  |  E  |
-------------------
|  E  |  E  |  E  |
###################
###################
|  X  |  O  |  E  |
-------------------
|  X  |  X  |  E  |
-------------------
|  E  |  E  |  O  |
###################


'Enter valid index'